<a href="https://colab.research.google.com/github/Yuns-u/Fine_Food_Summary/blob/main/Amazon_food_reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Amazone의 음식 리뷰들을 요약추출해본다.

앱스토어의 리뷰들을 추출요약하는 프로젝트를 해보려했으나 추출요약은 비지도학습을 위한 머신러닝모델을 만드는 것이 아니었다. 딥러닝을 보다 유용하게 사용해보기 위해서 생성요약을 하는 것을 통해 추후에 발전시키고자 한다.

일단, 생성요약(abstractive summarization)을 통해 원문에 없는 문장이라도 핵심 문맥을 반영하여 원문을 요약하는 인공 신경망을 구축해보고자 한다. 이를 위해서 keras에서 원문과 실제 요약문인 레이블 데이터를 가지고 있는 데이터를 확보했고 이를 통해 학습시키고자 한다.

In [ ]:
pip install nltk

In [ ]:
#필요한 모듈 불러오기
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import urllib.request

np.random.seed(seed=42)

In [ ]:
from google.colab import files
myfiles = files.upload()

# 데이터 불러오기

해당 데이터가 매우 크기 때문에 10만개의 row에 대해서 데이터를 불러오고자 한다.

In [ ]:
df = pd.read_csv('Reviews.csv', nrows=100000)
print('total reviews: ', len(df))

In [ ]:
df.head()

생성요약을 위해서 필요한 feature는 리뷰의 원문인 text와 그에 대한 요약문인 summary이다. 

따라서 필요한 데이터인 text와 summary로 해당 데이터프레임을 재구성하여 데이터 프레임 자체의 차원의 수를 줄이고자 한다.

In [ ]:
df = df[['Text','Summary']]
df.head()

# 데이터 전처리

In [ ]:
# 중복 확인
print('text열에서 중복이 아닌 데이터의 수: ', df['Text'].nunique())
print('summary 열에서 중복이 아닌 데이터의 수: ', df['Summary'].nunique())

요약문들은 짧다면 겹칠 가능성이 클 것이다.


원문들은 대체로 길이가 길기 때문에 겹칠 가능성은 요약문보다 적을 것이다. 

In [ ]:
#text열에서 중복제거(원본데이터에서의 중복치도 지워주었다.)
df.drop_duplicates(subset=['Text'], inplace=True)
print('전체 데이터의 수: ', len(df))

In [ ]:
#결측치 확인
df.isnull().sum()

In [ ]:
#결측치 처리
#하나뿐이므로 직접 줄여서 넣어주어도 좋을 것 같다.

#중복치가 없기 때문에 인덱스를 재설정해주는 것이 좋을 것이다.
df.reset_index(drop=True)
df.head()

In [ ]:
df.loc[df['Summary'].isna()]

In [ ]:
df['Text'].loc[33958]

임신을 해서 차를 마실 수 없다보니 두 세 개 밖에 안 먹었지만 나쁘지도 않고 좋지도 않았다고 한다. 따라서 Not a bad taste but not a big fan either로 요약하여 결측치를 대체하고자 한다.

In [ ]:
df['Summary'].loc[33958] = 'Not a bad taste but not a big fan of it either'
df['Summary'].loc[33958]

In [ ]:
df.loc[33958]

In [ ]:
#결측치 재확인
df.isnull().sum()

# 전처리

## 영문 약어를 정규화하기 위한 사전 만들기

아래의 함수는 다음의 링크를 참조하여 만들어졌다.
https://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python

In [ ]:
#전처리 함수 내 사용할 동의어 사전 만들기
contractions = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", 
                "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", 
                "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", 
                "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is", "I'd": "I would", 
                "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", 
                "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", 
                "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", 
                "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", 
                "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", 
                "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have",
                "o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", 
                "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", 
                "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", 
                "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", 
                "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", 
                "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is",
                "they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", 
                "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", 
                "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", 
                "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are", 
                "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", 
                "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", 
                "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", 
                "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", 
                "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have",
                "y'all're": "you all are","y'all've": "you all have", "you'd": "you would", "you'd've": "you would have", 
                "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"}

## 불용어 처리

In [ ]:
#NLTK의 불용어
import nltk
nltk.download("stopwords")

In [ ]:
stop_words = set(stopwords.words('english'))
print('불용어 개수: ',len(stop_words))
print(stop_words)

In [ ]:
# 전처리 함수 설계
def preprocess_sentence(sentence, remove_stopwords = True):
    sentence = sentence.lower() # 텍스트 소문자화
    sentence = BeautifulSoup(sentence, "lxml").text #html 태그 제거
    sentence = re.sub(r'\([^)]*\)', '', sentence) #괄호와 괄호 속의 문자열 제거
    sentence = re.sub('"','', sentence) #쌍따옴표 제거
    sentence = ' '.join([contractions[t] if t in contractions else t for t in sentence.split(" ")]) # 약어 정규화
    sentence = re.sub(r"'s\b","",sentence) # 소유격 's 제거
    sentence = re.sub("[^a-zA-Z]", " ", sentence) #영어 외 문자 공백
    sentence = re.sub('[m]{2,}', 'mm', sentence) # m이 3개 이상이면 2개로 변경. yummmmmmm->yumm

    # 불용어 제거 (Text)
    if remove_stopwords:
        tokens = ' '.join(word for word in sentence.split() if not word in stop_words if len(word) > 1)
    # 불용어 미제거 (Summary)
    else:
        tokens = ' '.join(word for word in sentence.split() if len(word) > 1)
    return tokens

In [ ]:
# Text열 전처리하기
cleaned_text = []

for stce in df['Text']:
  cleaned_text.append(preprocess_sentence(stce))

cleaned_text[:5]

In [ ]:
# Summary열 전처리하기
cleaned_summary = []

for s in df['Summary']:
  cleaned_summary.append(preprocess_sentence(s))

cleaned_summary[:5]

In [ ]:
# 전처리된 결과물들을 데이터프레임에 저장하기

df['Text'] = cleaned_text
df['Summary'] = cleaned_summary

In [ ]:
# 전처리 과정에서 빈 값이 생겼다면 Null로 변환한 뒤 확인하기
df.replace('', np.nan, inplace=True)
print(df.isnull().sum())

Summary 열에서 291개가 결측치가 있다. 결측치들을 제거한 뒤 전체 샘플 수를 확인해본다.

In [ ]:
df.dropna(axis=0, inplace=True)
print('전체 데이터수: ', len(df))

# 전처리된 텍스트들의 길이 분포 살펴보기



In [ ]:
# 길이 분포 출력
text_len = [len(s.split()) for s in df['Text']]
summ_len = [len(s.split()) for s in df['Summary']]

print('원문의 최소 길이: ', np.min(text_len))
print('원문의 최대 길이: ', np.max(text_len))
print('원문의 평균 길이: ', np.mean(text_len))

print('요약의 최소 길이: ', np.min(summ_len))
print('요약의 최대 길이: ', np.max(summ_len))
print('요약의 평균 길이: ', np.mean(summ_len))

In [ ]:
# 박스플롯으로 살펴보기

# 요약문 텍스트 길이 분포 박스플롯
plt.subplot(1,2,1)
plt.boxplot(summ_len)
plt.title('Text Length of Summary')
plt.show()

# 원문 텍스트 길이 분포 박스플롯
plt.subplot(1,2,1)
plt.boxplot(text_len)
plt.title('Text Length of Original Text')
plt.show()

In [ ]:
# 히스토그램으로 살펴보기

# 요약문 텍스트 길이 분포 히스토그램
plt.title('Text Length of Summary')
plt.hist(summ_len, bins=40)
plt.xlabel('length of samples')
plt.ylabel('the number of samples')
plt.show()

# 원문 텍스트 길이 분포 히스토그램
plt.title('Text Length of Original Text')
plt.hist(text_len, bins=40)
plt.xlabel('length of samples')
plt.ylabel('the number of samples')
plt.show()

원문 텍스트는 대체적으로 100이하의 길이를 가지며 평균적으로 38의 길이를 가지고 있다.

요약의 경우 대체적으로 15이하의 길이를 가지며 평균 길이는 4이다.
여기에서 패딩의 길이는 평균 길이보다 크게 잡아 60과 8로 정해주었다.


In [ ]:
text_max_len =60
summ_max_len =8

In [ ]:
def below_threshold_len(max_len, nested_list):
  cnt = 0
  for s in nested_list:
    if (len(s.split()) <= max_len):
      cnt = cnt+1
  print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (cnt/len(nested_list))))

In [ ]:
below_threshold_len(text_max_len, df['Text'])

In [ ]:
below_threshold_len(summ_max_len, df['Summary'])

text열은 길이가 60 이하인 경우가 83%이고 summary 열은 길이가 8 이하인 경우가 99%로 정해준 최대 길이보다 큰 샘플들은 연산의 수월성을 위하여 제거하고자 한다.

In [ ]:
df = df[df['Text'].apply(lambda x: len(x.split()) <= text_max_len)]
df

In [ ]:
df = df[df['Summary'].apply(lambda x: len(x.split()) <= summ_max_len)]
df

In [ ]:
print('전체 샘플수: ',len(df))

# seq2seq 훈련을 위한 준비
seq2seq 훈련을 하기 위해서는 디코더의 입력과 레이블에 시작 토큰과 종료 토큰을 추가해주어야 한다. 시작토큰을 'sijaktoken', 종료 토큰은 'jongryotoken'이라 만들어 앞뒤로 추가해주었다.

In [ ]:
#시작토큰과 종료토큰 앞뒤로 붙여주기
df['decoder_input'] = df['Summary'].apply(lambda x: 'sijaktoken '+x)
df['decoder_target'] = df['Summary'].apply(lambda x: x+' jongryotoken')

df.head()

In [ ]:
#인코더의 입력, 디코더의 입력과 레이블을 각각 정해주어 array로 만들어주기
encoder_input = np.array(df['Text'])
decoder_input = np.array(df['decoder_input'])
decoder_target = np.array(df['decoder_target'])

# 데이터 분리

훈련데이터와 테스트 데이터를 분리하기 위해 먼저 순서가 섞인 정수 시퀀스를 만들어준다.

In [ ]:
#순서를 섞어 정수 시퀀스를 만들어주기
indicies = np.arange(encoder_input.shape[0])
np.random.shuffle(indicies)
print(indicies)

이렇게 랜덤으로 섞인 시퀀스를 데이터의 샘플 순서로 정의하여 샘플의 순서를 섞이게 할 수 있다.

In [ ]:
encoder_input = encoder_input[indicies]
decoder_input = decoder_input[indicies]
decoder_target = decoder_target[indicies]

일반적으로 훈련데이터와 테스트데이터는 8:2로 나누므로 그렇게 나누어준다.
정제된 전체 데이터의 20%는 14673라고 할 수 있다.

In [ ]:
test_num = int(len(indicies)*0.2)
test_num

In [ ]:
train_encoder_input = encoder_input[:-test_num]
train_decoder_input = decoder_input[:-test_num]
train_decoder_target = decoder_target[:-test_num]

test_encoder_input = encoder_input[-test_num:]
test_decoder_input = decoder_input[-test_num:]
test_decoder_target = decoder_target[-test_num:]

In [ ]:
print('훈련데이터의 개수: ', len(train_encoder_input))
print('훈련레이블의 개수: ', len(train_decoder_input))
print('테스트데이터의 개수: ', len(test_encoder_input))
print('테스트레이블의 개수: ', len(test_decoder_input))

# 정수인코딩
텍스트를 숫자로 처리할 수 있도록 훈련 데이터와 테스트 데이터에 정수 인코딩을 해야한다.
훈련데이터에 대해서 단어 집합을 만들어야 하는데 먼저 원문에 해당하는 encoder_input_train에 대해서 진행해 단어집합의 크기를 어느 정도로 설정하면 좋을지 판단해볼 수 있다.

In [ ]:
src_tokenizer = Tokenizer()
src_tokenizer.fit_on_texts(train_encoder_input)

단어집합이 생성되면서 각 단어에 고유한 정수가 붙었다. 

각 단어의 고유 정수는 `src_tokenizer.word_index`에 저장되어 있다.

먼저 빈도수가 낮은 단어들은 자연어 처리에서 배제하고자 한다.
빈도수가 10 미만인 단어들이 이 데이터에서 얼마만큼의 비중을 차지하는지 확인해볼 수 있다.

In [ ]:
threshold = 10
total_cnt = len(src_tokenizer.word_index) #단어의 개수
rare_cnt = 0 #빈도수가 threshold보다 작은 개수를 센다.
total_freq = 0 #훈련데이터의 전체 단어 빈도수 총합
rare_freq = 0 #등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총합

#key와 value로  단어와 빈도수의 쌍(pair)을 만들어준다.
for key, value in src_tokenizer.word_counts.items():
  total_freq = total_freq+value

  #단어의 등장 빈도수가 threshold보다 작으면
  if(value < threshold):
    rare_cnt = rare_cnt+1
    rare_freq = rare_freq + value

print('단어집합(vocabulary)의 크기: ',total_cnt)
print('등장빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold-1, rare_cnt))
print('단어 집합에서 희귀 단어를 제외시킬 경우의 단어 집합의 크기 %s'%(total_cnt-rare_cnt))
print('단어집합에서 희귀 단어의 비율: ',(rare_cnt/total_cnt)*100)
#print('전체 등장빈도에서 희귀 단어 등장 빈도비율: ',(rare_freq/total_freq)*100)
print('전체 등장빈도에서 희귀 단어 등장 빈도비율: ',0.03874439358357905*100)

등장 빈도가 threshold 값인 10회 미만, 즉 9회 이하인 단어드은 단어집합에서 78%를 차지하지만 실제 훈련데이터에서 등장빈도로 차지하는 비중은 3.8% 정도 밖에 되지 않는다.

따라서 등장빈도가 10회 미만인 단어들은 정수 인코딩 과정에서 배제하고자 한다.
위와 비슷한 단어 집합의 크기는 7652이므로 단어집합의 크기를 7652로 설정하고자 한다.

In [ ]:
src_vocab = 7652
src_tokenizer = Tokenizer(num_words=src_vocab)
src_tokenizer.fit_on_texts(train_encoder_input)

#텍스트 시퀀스를 정수 시퀀스로 변환
train_encoder_input = src_tokenizer.texts_to_sequences(train_encoder_input)
test_encoder_input = src_tokenizer.texts_to_sequences(test_encoder_input)

In [ ]:
print(train_encoder_input[:3])

In [ ]:
#레이블에게도 텍스트 시퀀스를 정수시퀀스로 변환
tar_tokenizer = Tokenizer()
tar_tokenizer.fit_on_texts(train_decoder_input)

레이블에 해당하는 summary 데이터에도 단어집합이 생성되면서 각 단어에 고유한 정수가 부여되었다. 등장 빈도수가 6회 미만인 단어들이 이 데이터에서 얼마만큼의 비중을 차지하는지 다시 살펴보고자 한다.

In [ ]:
threshold = 6
total_cnt = len(tar_tokenizer.word_index) #단어의 개수
rare_cnt = 0 #빈도수가 threshold보다 작은 개수를 센다.
total_freq = 0 #훈련데이터의 전체 단어 빈도수 총합
rare_freq = 0 #등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총합

#key와 value로  단어와 빈도수의 쌍(pair)을 만들어준다.
for key, value in tar_tokenizer.word_counts.items():
  total_freq = total_freq+value

  #단어의 등장 빈도수가 threshold보다 작으면
  if(value < threshold):
    rare_cnt = rare_cnt+1
    rare_freq = rare_freq + value

print('단어집합(vocabulary)의 크기: ',total_cnt)
print('등장빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold-1, rare_cnt))
print('단어 집합에서 희귀 단어를 제외시킬 경우의 단어 집합의 크기 %s'%(total_cnt-rare_cnt))
print('단어집합에서 희귀 단어의 비율: ',(rare_cnt/total_cnt)*100)
#print('전체 등장빈도에서 희귀 단어 등장 빈도비율: ',(rare_freq/total_freq)*100)
print('전체 등장빈도에서 희귀 단어 등장 빈도비율: ',0.07630141772245003*100)

등장빈도가 5회 이하인 단어들은 단어 집합에서 약 77%를 차지하지만 실제로 훈련 데이터에서 등장 빈도로 차지하는 비중은 7%정도로 매우 낮다. 이 단어들을 정수 인코딩 과정에서 배제할 수 있다.

In [ ]:
tar_vocab = 2553
tar_tokenizer = Tokenizer(num_words=tar_vocab)
tar_tokenizer.fit_on_texts(train_decoder_input)
tar_tokenizer.fit_on_texts(train_decoder_target)

#텍스트 시퀀스를 정수 시퀀스로 변환
train_decoder_input = tar_tokenizer.texts_to_sequences(train_decoder_input)
train_decoder_target = tar_tokenizer.texts_to_sequences(train_decoder_target)
test_decoder_input = tar_tokenizer.texts_to_sequences(test_decoder_input)
test_decoder_target = tar_tokenizer.texts_to_sequences(test_decoder_target)

In [ ]:
#정수 인코딩이 잘 되었는지 훈련 데이터에 대해서 샘플 추출해서 확인하기
print(train_decoder_input[:3])
print(train_decoder_target[:3])

# 빈 샘플 제거
빈도수가 낮은 단어만으로 구성되었던 샘플은 이제 빈 샘플이 되었다.
이 현상은 길이가 상대적으로 짧았던 요약문의 경우 빈 샘플이 되어버린 경우가 많을 것이다.

요약문에서 0이 된 값들의 인덱스를 받아와서 삭제를 해줘야할 것이다.

이 때, 요약문인 decoder_input과 decoder_target에는 시작토큰과 종료토큰이 추가된 상태이다. 이 두 토큰은 모든 데이터에 있기 때문에 단어집합 제한에도 삭제되지 않는다. 따라서 길이가 0인 요약문의 실제 길이는 1이된다. 

In [ ]:
drop_train = [index for index, sentence in enumerate(train_decoder_input) if len(sentence) == 1]
drop_test = [index for index, sentence in enumerate(test_decoder_input) if len(sentence) == 1]

print('삭제할 훈련 데이터의 개수:',len(drop_train))
print('삭제할 테스트 데이터의 개수:',len(drop_test))

삭제해야하는 데이터들을 삭제하여 각각의 개수를 구하면 아래와 같다.

In [ ]:
train_encoder_input = np.delete(train_encoder_input, drop_train, axis=0)
train_decoder_input = np.delete(train_decoder_input, drop_train, axis=0)
train_decoder_target = np.delete(train_decoder_target, drop_train, axis=0)

test_encoder_input = np.delete(test_encoder_input, drop_test, axis=0)
test_decoder_input = np.delete(test_decoder_input, drop_test, axis=0)
test_decoder_target = np.delete(test_decoder_target, drop_test, axis=0)

print('훈련 데이터의 개수:',len(train_encoder_input))
print('훈련 레이블의 개수:',len(train_decoder_input))
print('테스트 데이터의 개수:',len(test_encoder_input))
print('테스트 레이블의 개수:',len(test_decoder_input))

# 패딩하기
앞의 최대 길이로 맞춰 훈련데이터와 테스트 데이터에 대해서 패딩을 하면 아래와 같다.

In [ ]:
train_encoder_input = pad_sequences(train_encoder_input, maxlen = text_max_len, padding='post')
test_encoder_input = pad_sequences(test_encoder_input, maxlen = text_max_len, padding='post')
train_decoder_input = pad_sequences(train_decoder_input, maxlen = summ_max_len, padding='post')
train_decoder_target = pad_sequences(train_decoder_target, maxlen = summ_max_len, padding='post')
test_decoder_input = pad_sequences(test_decoder_input, maxlen = summ_max_len, padding='post')
test_decoder_target = pad_sequences(test_decoder_target, maxlen = summ_max_len, padding='post')

# seq2seq와 attention으로 요약 모델 설계 및 훈련시키기

In [ ]:
#필요한 모듈 불러오기
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint 

## 인코더 설계
인코드는 LSTM을 3층을 쌓았다.

In [ ]:
embedding_dim = 128
hidden_size = 256

#인코더
encoder_inputs = Input(shape=(text_max_len,))

#인코더의 임베딩 층
enc_emb = Embedding(src_vocab, embedding_dim)(encoder_inputs)

#인코더의 LSTM 1층
encoder_lstm_1 = LSTM(hidden_size, return_sequences=True, return_state=True, dropout=0.4, recurrent_dropout=0.4)
encoder_output_1, state_h1, state_c1 = encoder_lstm_1(enc_emb)

#인코더의 LSTM 2층
encoder_lstm_2 = LSTM(hidden_size, return_sequences=True, return_state=True, dropout=0.4, recurrent_dropout=0.4)
encoder_output_2, state_h2, state_c2 = encoder_lstm_2(enc_emb)

#인코더의 LSTM 3층
encoder_lstm_3 = LSTM(hidden_size, return_sequences=True, return_state=True, dropout=0.4, recurrent_dropout=0.4)
encoder_outputs, state_h, state_c = encoder_lstm_3(enc_emb)

## 디코더 설계
디코더의 설계는 인코더와 사실상 동일하지만 초기상태를 인코더의 상태로 주어야한다.
출력층은 제외하고 설계한다.

In [ ]:
#디코더
decoder_inputs = Input(shape=(None,))

#디코더의 임베딩 층
dec_emb_layer= Embedding(tar_vocab, embedding_dim)
dec_emb = dec_emb_layer(decoder_inputs)

#디코더의 LSTM
decoder_lstm = LSTM(hidden_size, return_sequences=True, return_state=True, dropout=0.4, recurrent_dropout=0.2)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=[state_h, state_c])

디코더 출력층 설계

In [ ]:
#디코더의 출력층
decoder_softmax_layer = Dense(tar_vocab, activation='softmax')
decoder_softmax_outputs = decoder_softmax_layer(decoder_outputs)

# 모델 설계

In [ ]:
#모델 정의
model = Model([encoder_inputs, decoder_inputs], decoder_softmax_outputs)
model.summary()

In [ ]:
#위의 모델을 compile하여 써보고자 한다.
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')

조기종료조건을 설정하고 모델을 학습시키면 아래와 같다.

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience = 2)
history = model.fit(x = [train_encoder_input, train_decoder_input], y = train_decoder_target, \
          validation_data = ([test_encoder_input, test_decoder_input], test_decoder_target),
          batch_size = 256, callbacks=[es], epochs = 20)

학습 과정에서 기록된 훈련 데이터의 손실과 테스트 데이터의 손실 히스토리를 시각화하면 아래와 같다.

In [ ]:
plt.plot(history.history['loss'],label='train')
plt.plot(history.history['val_loss'],label='test')
plt.xlabel('number of epochs')
plt.ylabel('value of loss')
plt.legend()
plt.show()

epoch 5 정도에서 손실이 가장 적은 것을 알 수 있다. 

# -------------확인----------------

In [ ]:
src_index_to_word = src_tokenizer.index_word # 원문 단어 집합에서 정수 -> 단어를 얻음
tar_word_to_index = tar_tokenizer.word_index # 요약 단어 집합에서 단어 -> 정수를 얻음
tar_index_to_word = tar_tokenizer.index_word # 요약 단어 집합에서 정수 -> 단어를 얻음

In [ ]:
# 원문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2text(input_seq):
    sentence=''
    for i in input_seq:
        if(i!=0):
            sentence = sentence + src_index_to_word[i]+' '
    return sentence

# 요약문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2summary(input_seq):
    sentence=''
    for i in input_seq:
        if((i!=0 and i!=tar_word_to_index['sijaktoken']) and i!=tar_word_to_index['jongryotoken']):
            sentence = sentence + tar_index_to_word[i] + ' '
    return sentence

In [ ]:
# 인코더 설계
encoder_model = Model(inputs=encoder_inputs, outputs=[encoder_outputs, state_h, state_c])

In [ ]:
# 이전 시점의 상태들을 저장하는 텐서
decoder_state_input_h = Input(shape=(hidden_size,))
decoder_state_input_c = Input(shape=(hidden_size,))

dec_emb2 = dec_emb_layer(decoder_inputs)
# 문장의 다음 단어를 예측하기 위해서 초기 상태(initial_state)를 이전 시점의 상태로 사용. 이는 뒤의 함수 decode_sequence()에 구현
# 훈련 과정에서와 달리 LSTM의 리턴하는 은닉 상태와 셀 상태인 state_h와 state_c를 버리지 않음.
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=[decoder_state_input_h, decoder_state_input_c])

In [ ]:
# 어텐션 함수
#decoder_hidden_state_input = Input(shape=(text_max_len, hidden_size))
#attn_out_inf, attn_states_inf = attn_layer([decoder_hidden_state_input, decoder_outputs2])
#decoder_inf_concat = Concatenate(axis=-1, name='concat')([decoder_outputs2, attn_out_inf])

#어텐션 함수 적용 전 디코더의 출력층
#decoder_softmax_layer = Dense(tar_vocab, activation='softmax')
#decoder_softmax_outputs = decoder_softmax_layer(decoder_outputs)

# 디코더의 출력층
decoder_softmax_layer = Dense(tar_vocab, activation='softmax')
decoder_softmax_outputs = decoder_softmax_layer(decoder_outputs) 

# 최종 디코더 모델
decoder_model = Model(
    [decoder_inputs] + [decoder_hidden_state_input,decoder_state_input_h, decoder_state_input_c],
    [decoder_outputs2] + [state_h2, state_c2])

---------------

In [ ]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    e_out, e_h, e_c = encoder_model.predict(input_seq)

     # <SOS>에 해당하는 토큰 생성
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = tar_word_to_index['sijaktoken']

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition: # stop_condition이 True가 될 때까지 루프 반복

        output_tokens, h, c = decoder_model.predict([target_seq] + [e_out, e_h, e_c])
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = tar_index_to_word[sampled_token_index]

        if(sampled_token!='jongryotoken'):
            decoded_sentence += ' '+sampled_token

        #  <eos>에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_token == 'jongryotoken'  or len(decoded_sentence.split()) >= (summ_max_len-1)):
            stop_condition = True

        # 길이가 1인 타겟 시퀀스를 업데이트
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # 상태를 업데이트 합니다.
        e_h, e_c = h, c

    return decoded_sentence

In [ ]:
for i in range(500, 510):
    print("원문 : ",seq2text(test_encoder_input[i]))
    print("실제 요약문 :",seq2summary(test_decoder_input[i]))
    print("예측 요약문 :",decode_sequence(test_encoder_input[i].reshape(1, text_max_len)))
    print("\n")

# ---------아직 이해 및 적용 안됨------------

# 어텐션 매커니즘 사용해보기
어텐션 메커니즘을 사용하기 위해서는 설꼐한 출력층을 사용하지 않고 어텐션 메커니즘이 결합된 새로운 출력층을 만들어 볼 수 있다. 어텐션 함수를 직접 작성하지 않고 깃허브에 공개된 함수를 사용할 수 있어서 아래의 코드를 통해 attention.py를 다운로드하고 attention layer를 불러올수 있다. 해당 어텐션은 바다나우 어텐션이다.

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/thushv89/attention_keras/master/src/layers/attention.py", filename="attention.py")
from attention import AttentionLayer

In [ ]:
#어텐션 층(어텐션 함수)
attn_layer = AttentionLayer(name='attention_layer')
attn_out, attn_states = attn_layer([encoder_outputs, decoder_outputs])

#어텐션의 결과와 디코더의 hidden state들을 연결
decoder_concat_input = Concatenate(axis=-1, name='concat_layer')([decoder_outputs, attn_out])

#디코더의 출력층
decoder_softmax_layer = Dense(tar_vocab, activation='softmax')
decoder_softmax_outputs = decoder_softmax_layer(decoder_concat_input)

#모델 정의
model = Model([encoder_inputs, decoder_inputs], decoder_softmax_outputs)
model.summary()